In [1]:
import numpy as np
import cv2 as cv
import os
import glob
from scipy.linalg import rq

## After calibration

In [2]:
P_rec = np.array("7.070493e+02 0.000000e+00 6.040814e+02 0.000000e+00 0.000000e+00 7.070493e+02 1.805066e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00".split()).astype(float).reshape(3, 4)

In [3]:
K_no_rec = np.array("9.812178e+02 0.000000e+00 6.900000e+02 0.000000e+00 9.758994e+02 2.471364e+02 0.000000e+00 0.000000e+00 1.000000e+00".split()).astype(float).reshape(3, 3)

In [4]:
distCoeffs = np.array("-3.791375e-01 2.148119e-01 1.227094e-03 2.343833e-03 -7.910379e-02".split()).astype(float).reshape(1, 5)

In [5]:
P_rec

array([[707.0493,   0.    , 604.0814,   0.    ],
       [  0.    , 707.0493, 180.5066,   0.    ],
       [  0.    ,   0.    ,   1.    ,   0.    ]])

In [6]:
K_no_rec

array([[981.2178,   0.    , 690.    ],
       [  0.    , 975.8994, 247.1364],
       [  0.    ,   0.    ,   1.    ]])

In [7]:
distCoeffs

array([[-0.3791375 ,  0.2148119 ,  0.00122709,  0.00234383, -0.07910379]])

In [8]:
path = 'results/kitti_test_output/0000000001_0000000002_matches.npz'

In [9]:
npz = np.load(path)
print(f"{npz.files=}")

print(f"{npz['keypoints0'].shape=}")
print(f"{npz['keypoints1'].shape=}")

print(f"{npz['matches'].shape=}")

print(f"{np.sum(npz['matches']>-1)=}")

print(f"{npz['match_confidence'].shape=}")

npz.files=['keypoints0', 'keypoints1', 'matches', 'match_confidence']
npz['keypoints0'].shape=(878, 2)
npz['keypoints1'].shape=(841, 2)
npz['matches'].shape=(878,)
np.sum(npz['matches']>-1)=666
npz['match_confidence'].shape=(878,)


In [10]:
match_indices = npz['matches'][npz['matches'] > -1]

points1 = npz['keypoints0'][npz['matches'] > -1]
points2 = npz['keypoints1'][match_indices]

In [11]:
def get_rbt_estimation(points1, points2, K, distCoeffs):
    pts_l_norm = cv.undistortPoints(np.expand_dims(points1, axis=1), cameraMatrix=K, distCoeffs=distCoeffs)
    pts_r_norm = cv.undistortPoints(np.expand_dims(points2, axis=1), cameraMatrix=K, distCoeffs=distCoeffs)
    
    E, mask = cv.findEssentialMat(pts_l_norm, pts_r_norm, focal=1.0, pp=(0., 0.), method=cv.RANSAC, prob=0.999, threshold=3.0)
    points, R_hat, t_hat, mask = cv.recoverPose(E, pts_l_norm, pts_r_norm)
    
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.array([0., 0., 0., 1.])))
    
    
    return R_hat, t_hat, T_hat

In [12]:
get_rbt_estimation(points1, points2, K_no_rec, distCoeffs)

(array([[ 0.99920467,  0.03200302,  0.02378717],
        [-0.0317425 ,  0.99943276, -0.01125018],
        [-0.02413372,  0.01048617,  0.99965374]]),
 array([[ 0.10580275],
        [-0.08789065],
        [-0.99049534]]),
 array([[ 0.99920467,  0.03200302,  0.02378717,  0.10580275],
        [-0.0317425 ,  0.99943276, -0.01125018, -0.08789065],
        [-0.02413372,  0.01048617,  0.99965374, -0.99049534],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]))

In [ ]:
# Normalize for Esential Matrix calaculation
pts_l_norm = cv.undistortPoints(np.expand_dims(points1, axis=1), cameraMatrix=K_no_rec, distCoeffs=distCoeffs)
pts_r_norm = cv.undistortPoints(np.expand_dims(points2, axis=1), cameraMatrix=K_no_rec, distCoeffs=distCoeffs)

In [43]:
E, mask = cv.findEssentialMat(pts_l_norm, pts_r_norm, focal=1.0, pp=(0., 0.), method=cv.RANSAC, prob=0.999, threshold=3.0)
points, R_hat, t_hat, mask = cv.recoverPose(E, pts_l_norm, pts_r_norm)

In [44]:
T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.array([0., 0., 0., 1.])))
T_hat

array([[ 0.8980957 , -0.02313778,  0.43919103, -0.38393758],
       [ 0.419907  ,  0.34205181, -0.84064182,  0.76302156],
       [-0.13077551,  0.93939619,  0.31691098,  0.51999042],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [35]:
M_r = np.hstack((R, t))
M_l = np.hstack((np.eye(3, 3), np.zeros((3, 1))))

P_l = np.dot(K_no_rec,  M_l)
P_r = np.dot(K_no_rec,  M_r)
point_4d_hom = cv.triangulatePoints(P_l, P_r, np.expand_dims(points1, axis=1), np.expand_dims(points2, axis=1))
point_4d = point_4d_hom / np.tile(point_4d_hom[-1, :], (4, 1))
point_3d = point_4d[:3, :].T